In [1]:
import scrapy
import scrapy.crawler as crawler
from multiprocessing import Process, Queue
from twisted.internet import reactor
import json
import time
import unidecode

import pandas as pd

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

In [2]:
#load romes
#Load departments

In [106]:
departments = pd.read_csv('data/department code avg.csv')
#departments = departments[1:]
departments.head()

,code,department,cp
0,1,Ain,01160
1,2,Aisne,02000
2,3,Allier,03240
3,4,Alpes-de-Haute-Provence,04000
4,5,Hautes-Alpes,05260


In [107]:
#[cp for cp in departments[departments.cp.notna()]['cp']]

In [111]:
def get_value(df, column_key, column_value, key):
    try:
        return df[df[column_key].str.startswith(key)][column_value].values[0]
    except Exception as e:
        print(e)
        return ''
    
get_value(departments, 'department', 'code', 'Paris')

'75'

In [112]:
rome_codes = pd.DataFrame()
codes = ['M1706', 'D1509']
postes = ['promotion des ventes', 'responsable de secteur']

rome_codes['Intitulé du poste'] = postes
rome_codes['Code ROME'] = codes
rome_codes

,Intitulé du poste,Code ROME
0,promotion des ventes,M1706
1,responsable de secteur,D1509


In [113]:
#rome_codes = pd.read_csv('data/CODES ROME - Sheet1.csv')
#rome_codes

In [25]:
base_url = 'https://labonneboite.pole-emploi.fr/'

In [26]:
options = Options()
driver = webdriver.Firefox(options=options, executable_path=r'C:/driver/geckodriver.exe')

In [114]:
driver.get(base_url)

In [115]:
test_cp_code = '02000'
test_rome_code = 'M1706'

In [117]:
search_elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "j")))
search_elem.click()
search_elem.clear()
search_elem.send_keys(test_rome_code)

address_elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "l")))
address_elem.click()
address_elem.clear()
address_elem.send_keys(test_cp_code)

address_elem.send_keys(Keys.ENTER)

In [118]:
driver.execute_script("window.scrollTo(0, Math.ceil(document.body.scrollHeight/5));")

#100km
distance = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "d-3")))
distance.click()


In [119]:
driver.get(str(driver.current_url)+'&from=1&to=999')

In [120]:
companies = driver.find_elements_by_class_name('lbb-bright-container')

In [ ]:
results = []

for company in companies:
    try:
        h3 = company.find_element_by_tag_name('h3')
    except:
        h3 = None
        
    if h3 != None:
        #print(company.text)
        #print(company.find_element_by_class_name('rating-value').text)
        
        try:
            company_name = h3.text.split('-')[0].rstrip()
        except:
            company_name = None

        try:
            location = h3.text.split('-')[1].lstrip()
        except:
            location = None
            
        try:
            num_employees = company.find_element_by_class_name('grid-col-8').find_element_by_tag_name('p').text
        except:
            num_employees = None
            
        try:
            distance = str(company.find_element_by_class_name('travel-distance-duration').text).replace(' km de votre lieu de recherche', '')
        except:
            distance = None
        
        
        #try:
        #    distance_time = str(company.find_element_by_class_name('travel-duration').text).replace(' min en voiture', '')
        #except:
        #    distance_time = None
            
        #try:
        #    distance = distance.split(' ')[0]
        #except:
        #    pass
        
        
        
        try:
            rating = company.find_element_by_class_name('rating-value').text
        except:
            rating = None

        results.append(
            {
                'jobs': get_value(rome_codes, 'Code ROME', 'Intitulé du poste', test_rome_code), 
                'rome_code': test_rome_code, 
                'company_name': company_name,
                'location': location,
                'num_employees': num_employees,
                'distance': distance,
                #'distance_time': distance_time,
                'rating': rating,
                'cp': cp,
                'department_code': get_value(departments[departments.cp.notna()], 'code', 'department', cp)
            }
        )
    
results

In [144]:
departments[departments.cp.notna()]['cp'][:1]

0    01160
Name: cp, dtype: object

In [157]:
#phase2
import time

results = []

for cp in departments[departments.cp.notna()]['cp']:
    print('Scraping', get_value(departments[departments.cp.notna()], 'cp', 'department', cp))
    for code in rome_codes['Code ROME']:

        print('Scrapping: ', code)

        try:
            driver.get(base_url)

            search_elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "j")))
            search_elem.click()
            search_elem.clear()
            search_elem.send_keys(code)

            address_elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "l")))
            address_elem.click()
            address_elem.clear()
            address_elem.send_keys(cp)

            address_elem.send_keys(Keys.ENTER)

            driver.execute_script("window.scrollTo(0, Math.ceil(document.body.scrollHeight/5));")

            #100km
            distance = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "d-3")))
            distance.click()

            driver.get(str(driver.current_url)+'&from=1&to=999')
            companies = driver.find_elements_by_class_name('lbb-bright-container')

            for company in companies:
                try:
                    h3 = company.find_element_by_tag_name('h3')
                except:
                    h3 = None

                if h3 != None:
                    try:
                        company_name = h3.text.split('-')[0].rstrip()
                    except:
                        company_name = None

                    try:
                        location = h3.text.split('-')[1].lstrip()
                    except:
                        location = None

                    try:
                        num_employees = company.find_element_by_class_name('grid-col-8').find_element_by_tag_name('p').text
                    except:
                        num_employees = None

                    try:
                        distance = str(company.find_element_by_class_name('travel-distance-duration').text).replace(' km de votre lieu de recherche', '')
                    except:
                        distance = None
                    try:
                        rating = company.find_element_by_class_name('rating-value').text
                    except:
                        rating = None

                    results.append(
                        {
                            'jobs': get_value(rome_codes, 'Code ROME', 'Intitulé du poste', code), 
                            'rome_code': code, 
                            'company_name': company_name,
                            'location': location,
                            'num_employees': num_employees,
                            'distance': distance,
                            #'distance_time': distance_time,
                            'rating': rating,
                            'cp': cp,
                            'department': get_value(departments[departments.cp.notna()], 'cp', 'department', cp)
                        }
                    )
        except:
            pass

        time.sleep(10)
    
results

Scraping Ain
Scrapping:  M1706
Scrapping:  D1509
Scraping Aisne
Scrapping:  M1706
Scrapping:  D1509
Scraping Allier
Scrapping:  M1706
Scrapping:  D1509
Scraping Alpes-de-Haute-Provence
Scrapping:  M1706
Scrapping:  D1509
Scraping Hautes-Alpes
Scrapping:  M1706
Scrapping:  D1509
Scraping Alpes-Maritimes
Scrapping:  M1706
Scrapping:  D1509
Scraping Ardèche
Scrapping:  M1706
Scrapping:  D1509
Scraping Ardennes
Scrapping:  M1706
Scrapping:  D1509
Scraping Ariège
Scrapping:  M1706
Scrapping:  D1509
Scraping Aube
Scrapping:  M1706
Scrapping:  D1509
Scraping Aude
Scrapping:  M1706
Scrapping:  D1509
Scraping Aveyron
Scrapping:  M1706
Scrapping:  D1509
Scraping Bouches-du-Rhône
Scrapping:  M1706
Scrapping:  D1509
Scraping Calvados
Scrapping:  M1706
Scrapping:  D1509
Scraping Cantal
Scrapping:  M1706
Scrapping:  D1509
Scraping Charente
Scrapping:  M1706
Scrapping:  D1509
Scraping Charente-Maritime
Scrapping:  M1706
Scrapping:  D1509
Scraping Cher
Scrapping:  M1706
Scrapping:  D1509
Scraping Corr

[{'jobs': 'promotion des ventes',
  'rome_code': 'M1706',
  'company_name': 'ADREXO',
  'location': 'HAUTECOURT',
  'num_employees': '100 à 199 salariés',
  'distance': '12.5',
  'rating': '2.5',
  'cp': '01160',
  'department': 'Ain'},
 {'jobs': 'promotion des ventes',
  'rome_code': 'M1706',
  'company_name': 'MEDIAPOST',
  'location': 'VAULX',
  'num_employees': '100 à 199 salariés',
  'distance': '47.0',
  'rating': '2.8',
  'cp': '01160',
  'department': 'Ain'},
 {'jobs': 'promotion des ventes',
  'rome_code': 'M1706',
  'company_name': 'STRADA ACTION',
  'location': 'LYON',
  'num_employees': '0 salarié',
  'distance': '49.5',
  'rating': '2.5',
  'cp': '01160',
  'department': 'Ain'},
 {'jobs': 'promotion des ventes',
  'rome_code': 'M1706',
  'company_name': 'ORANGINA SUNTORY FRANCE PRODUCTION',
  'location': 'MEYZIEU',
  'num_employees': '100 à 199 salariés',
  'distance': '40.8',
  'rating': '3.0',
  'cp': '01160',
  'department': 'Ain'},
 {'jobs': 'promotion des ventes',
  '

In [151]:
#phase1
'''
import time

results = []

for code in rome_codes['Code ROME']:
    
    print('Scrapping: ', code)
    
    try:

        driver.get(base_url)

        search_elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "j")))
        search_elem.click()
        search_elem.clear()
        search_elem.send_keys(code)

        address_elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "l")))
        address_elem.click()
        address_elem.clear()
        address_elem.send_keys("Villecomtal-sur-Arros 32730")

        address_elem.send_keys(Keys.ENTER)

        driver.execute_script("window.scrollTo(0, Math.ceil(document.body.scrollHeight/5));")

        #100km
        distance = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "d-4")))
        distance.click()

        driver.get(str(driver.current_url)+'&from=1&to=999')
        companies = driver.find_elements_by_class_name('lbb-bright-container')

        for company in companies:
            try:
                h3 = company.find_element_by_tag_name('h3')
            except:
                h3 = None

            if h3 != None:
                #print(company.text)
                #print(company.find_element_by_class_name('rating-value').text)

                try:
                    company_name = h3.text.split('-')[0].rstrip()
                except:
                    company_name = None

                try:
                    location = h3.text.split('-')[1].lstrip()
                except:
                    location = None

                try:
                    num_employees = company.find_element_by_class_name('grid-col-8').find_element_by_tag_name('p').text
                except:
                    num_employees = None

                try:
                    distance = str(company.find_element_by_class_name('travel-distance-duration').text).replace(' km de votre lieu de recherche', '')
                except:
                    distance = None

                
                #try:
                #    distance_time = str(company.find_element_by_class_name('travel-duration').text).replace(' min en voiture', '')
                #except:
                #    distance_time = None

                #try:
                #    distance = distance.split(' ')[0]
                #except:
                #    pass
                

                try:
                    rating = company.find_element_by_class_name('rating-value').text
                except:
                    rating = None

                results.append(
                    {
                        'jobs': rome_codes[rome_codes['Code ROME'].str.startswith(code)]['Intitulé du poste '].values[0], 
                        'rome_code': code, 
                        'company_name': company_name,
                        'location': location,
                        'num_employees': num_employees,
                        'distance': distance,
                        #'distance_time': distance_time,
                        'rating': rating
                    }
                )
    except:
        pass
    
    time.sleep(10)
    
results
'''

'\nimport time\n\nresults = []\n\nfor code in rome_codes[\'Code ROME\']:\n    \n    print(\'Scrapping: \', code)\n    \n    try:\n\n        driver.get(base_url)\n\n        search_elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "j")))\n        search_elem.click()\n        search_elem.clear()\n        search_elem.send_keys(code)\n\n        address_elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "l")))\n        address_elem.click()\n        address_elem.clear()\n        address_elem.send_keys("Villecomtal-sur-Arros 32730")\n\n        address_elem.send_keys(Keys.ENTER)\n\n        driver.execute_script("window.scrollTo(0, Math.ceil(document.body.scrollHeight/5));")\n\n        #100km\n        distance = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "d-4")))\n        distance.click()\n\n        driver.get(str(driver.current_url)+\'&from=1&to=999\')\n        companies = driver.find_elements_by_class_name(\'lbb

In [158]:
#pd.DataFrame(results).to_csv('data/labonneboite_phase2.csv', index=False)
pd.DataFrame(results)

,company_name,cp,department,distance,jobs,location,num_employees,rating,rome_code
0,ADREXO,01160,Ain,12.5,promotion des ventes,HAUTECOURT,100 à 199 salariés,2.5,M1706
1,MEDIAPOST,01160,Ain,47.0,promotion des ventes,VAULX,100 à 199 salariés,2.8,M1706
2,STRADA ACTION,01160,Ain,49.5,promotion des ventes,LYON,0 salarié,2.5,M1706
3,ORANGINA SUNTORY FRANCE PRODUCTION,01160,Ain,40.8,promotion des ventes,MEYZIEU,100 à 199 salariés,3.0,M1706
4,KARIO,01160,Ain,42.7,promotion des ventes,BELLEY,6 à 9 salariés,2.5,M1706
5,STRADA PERFORMANCE,01160,Ain,49.5 38 min en voiture,promotion des ventes,LYON,0 salarié,2.8,M1706
6,SAS HOLDIS,01160,Ain,37.4,responsable de secteur,LECLERC,0 salarié,2.5,D1509
7,CENTRE LECLERC,01160,Ain,44.4,responsable de secteur,GENAY,200 à 249 salariés,2.5,D1509
8,INTERMARCHE JEAN DE PARIS,01160,Ain,11.1,responsable de secteur,AMBERIEU,100 à 199 salariés,2.5,D1509
9,AUCHAN,01160,Ain,46.9,responsable de secteur,CALUIRE,500 à 999 salariés,2.7,D1509


In [159]:
pd.DataFrame(results).to_clipboard(index=False)

In [136]:
df = pd.read_csv('data/labonneboite.csv')
df

,company_name,distance,jobs,location,num_employees,rating,rome_code
0,AXE TRAVAIL TEMPORAIRE,42.0 48 min en voiture,Adjointe RH,AUCH,1 ou 2 salariés,2.5,M1501
1,O2,78.6 89 min en voiture,Adjointe RH,MONT,0 salarié,2.5,M1501
2,MAIRIE,100.0 105 min en voiture,Adjointe RH,BLAGNAC,0 salarié,4.2,M1501
3,MANPOWER FRANCE,93.5 112 min en voiture,Adjointe RH,BOE,3 à 5 salariés,4.4,M1501
4,SDIS64,45.1 57 min en voiture,Adjointe RH,PAU,0 salarié,4.4,M1501
5,RANDSTAD,91.4 91 min en voiture,Adjointe RH,MURET,0 salarié,4.5,M1501
6,AQUILA RH,47.4 58 min en voiture,Adjointe RH,SERRES,1 ou 2 salariés,3.8,M1501
7,MAIRIE,47.9 59 min en voiture,Adjointe RH,PAU,1 000 à 1 999 salariés,3.9,M1501
8,LE&C GS,97.2 101 min en voiture,Adjointe RH,TOULOUSE,1 000 à 1 999 salariés,4.4,M1501
9,CA PAU BEARN PYRENEES,47.8 59 min en voiture,Adjointe RH,PAU,0 salarié,4.2,M1501


In [137]:
df['distance'][1].split(' ')[0]

'78.6'

In [138]:
'98.2'.split(' ')[0]

'98.2'

In [139]:
df['distance'] = [distance.split(' ')[0] for distance in df['distance']]

In [130]:
df.to_csv('data/labonneboite_parsed.csv', index=False)

In [142]:
df.to_clipboard()

In [140]:
df['num_employees'] = [num_employees.replace('1 000', '1000').replace('10 000', '10000').replace('2 000', '2000').replace('5 000', '5000').split(' ')[0] for num_employees in df['num_employees']]

In [141]:
df

,company_name,distance,jobs,location,num_employees,rating,rome_code
0,AXE TRAVAIL TEMPORAIRE,42.0,Adjointe RH,AUCH,1,2.5,M1501
1,O2,78.6,Adjointe RH,MONT,0,2.5,M1501
2,MAIRIE,100.0,Adjointe RH,BLAGNAC,0,4.2,M1501
3,MANPOWER FRANCE,93.5,Adjointe RH,BOE,3,4.4,M1501
4,SDIS64,45.1,Adjointe RH,PAU,0,4.4,M1501
5,RANDSTAD,91.4,Adjointe RH,MURET,0,4.5,M1501
6,AQUILA RH,47.4,Adjointe RH,SERRES,1,3.8,M1501
7,MAIRIE,47.9,Adjointe RH,PAU,1000,3.9,M1501
8,LE&C GS,97.2,Adjointe RH,TOULOUSE,1000,4.4,M1501
9,CA PAU BEARN PYRENEES,47.8,Adjointe RH,PAU,0,4.2,M1501
